In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
# Loading the dataset
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [4]:
ds['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [5]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

# WITHOUT FINE-TUNING

In [6]:
!pip install transformers

In [7]:
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
article_1 = ds['train'][1]['dialogue']

In [9]:
pipe(article_1, max_length = 20, min_length = 10, do_sample = False)

[{'summary_text': 'Ricky has received his Polio, Tetanus and Hepatitis B shots.'}]

# WITH FINE-TUNING

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [11]:
# tokenization
def preprocessor_function(batch):
  source = batch['dialogue']
  target = batch['summary']
  source_ids = tokenizer(source, truncation = True, padding = 'max_length', max_length = 128)
  target_ids = tokenizer(target, truncation = True, padding = 'max_length', max_length = 128)

  labels = target_ids['input_ids']
  labels = [[(label if label != tokenizer.pad_token_id else - 100) for label in labels_example] for labels_example in labels]

  return {
      "input_ids": source_ids["input_ids"],
      "attention_mask": source_ids["attention_mask"],
      "labels": labels
  }

In [12]:
df_source = ds.map(preprocessor_function, batched = True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [13]:
import os
os.environ['WANDB_DISABLED'] = 'True'

In [14]:
# training_arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "/content",
    per_device_train_batch_size = 8,
    num_train_epochs = 2,
    remove_unused_columns = True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = df_source['train'],
    eval_dataset = df_source['test']
)

In [16]:
trainer.train()

Step,Training Loss
500,1.591300
1000,1.490500
1500,1.435400
2000,1.085100
2500,1.020500
3000,1.003800


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=1.2623270962755548, metrics={'train_runtime': 3023.0017, 'train_samples_per_second': 8.243, 'train_steps_per_second': 1.031, 'total_flos': 6750530835578880.0, 'train_loss': 1.2623270962755548, 'epoch': 2.0})

In [17]:
eval_results = trainer.evaluate()

In [18]:
eval_results

{'eval_loss': 1.6603035926818848,
 'eval_runtime': 48.491,
 'eval_samples_per_second': 30.934,
 'eval_steps_per_second': 3.877,
 'epoch': 2.0}

# Saving the model

In [19]:
model.save_pretrained("/content/model_directory")
tokenizer.save_pretrained("/content/model_directory")

('/content/model_directory/tokenizer_config.json',
 '/content/model_directory/special_tokens_map.json',
 '/content/model_directory/vocab.json',
 '/content/model_directory/merges.txt',
 '/content/model_directory/added_tokens.json',
 '/content/model_directory/tokenizer.json')

In [33]:
tokenizer = AutoTokenizer.from_pretrained("/content/model_directory")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/model_directory")

In [53]:
def summarize(blog_post):
    # Tokenize the input blog post
    inputs = tokenizer(blog_post, max_length=1024, truncation=True, return_tensors='pt')

    # Debug: Print token count
    print(f"Input token count: {len(inputs['input_ids'][0])}")

    # Generate the summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Debug: Print generated token count
    print(f"Generated token count: {len(summary_ids[0])}")

    # decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Debug: Print raw summary with special tokens
    raw_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=False)
    print(f"Raw summary (with special tokens): {raw_summary}")

    return summary

In [55]:
blog_post = """
# The Future of Remote Work: Balancing Flexibility and Collaboration

The COVID-19 pandemic forced an unprecedented shift to remote work, transforming what was once considered a privilege into a necessity overnight. As we move beyond the immediate crisis, organizations worldwide are reimagining the future of work. This evolution isn't simply about deciding between in-office or remote work—it's about creating sustainable models that balance employee flexibility with organizational needs.

## Remote Work: The New Normal

Recent studies suggest that approximately 25% of all professional jobs in North America will be remote by the end of 2022, with that number continuing to grow through 2023. This shift represents more than a temporary adjustment; it signals a fundamental change in how we conceptualize work.

The benefits for employees are well-documented: eliminated commutes, improved work-life balance, and the ability to work from anywhere. Companies have discovered their own advantages, from reduced office costs to access to global talent pools previously limited by geography. Productivity, contrary to early fears, has generally increased, though with important nuances across different roles and industries.

## Challenges in the Distributed Workforce

Despite these benefits, fully remote environments present significant challenges. Zoom fatigue—that unique exhaustion that comes from video conferencing—has become a recognized phenomenon. Many workers report difficulty in maintaining boundaries between work and personal life, with workdays often extending beyond traditional hours.

Perhaps most concerning is the impact on organizational culture and spontaneous collaboration. The casual conversations by the coffee machine, impromptu brainstorming sessions, and organic mentoring that occur in physical workspaces are difficult to replicate virtually. New employees particularly struggle to absorb company culture and build relationships in fully remote settings.

## Hybrid Models: The Emerging Compromise

In response, many organizations are adopting hybrid work models that combine remote flexibility with in-person collaboration. These approaches vary widely:

1. **Designated in-office days**: Teams coordinate to be present on specific days, maximizing collaborative opportunities.
2. **Activity-based planning**: Work location decisions based on the nature of tasks, with collaborative activities scheduled for office days.
3. **Hub-and-spoke models**: Organizations maintain smaller satellite offices closer to where employees live, reducing commutes while preserving in-person options.

Google's approach exemplifies this hybrid evolution. Their "flexible workweek" asks employees to be in the office three days per week, with teams deciding which days work best for their collaborative needs. They've also redesigned office spaces to support team collaboration rather than individual work.

## Technology Enabling the Transition

Technology companies are racing to fill the gaps in the remote work experience. Virtual reality meeting spaces, digital whiteboards, and asynchronous collaboration tools are evolving rapidly. These innovations aim to overcome the limitations of current videoconferencing platforms that inadequately replicate in-person interactions.

Microsoft's Mesh platform, for example, allows people in different physical locations to join collaborative experiences with the presence of being together. Similarly, startups like Teamflow are creating virtual offices where employees can "see" colleagues and spontaneously connect, mimicking the casual interactions of physical spaces.

## The Path Forward: Intentional Design

As we navigate this transformation, the most successful organizations will be those that approach remote and hybrid work with intentionality. Rather than simply replicating office-based processes online, they will reimagine workflows, communication patterns, and collaboration methods for distributed teams.

The future of work requires balancing the autonomy employees have grown accustomed to with the collaborative culture that drives innovation. Organizations that can strike this balance—providing flexibility while fostering meaningful connection—will create sustainable models that attract and retain top talent in the evolving workplace landscape.

The coming years will likely see continued experimentation as organizations learn what works best for their unique contexts. What's clear is that there's no returning to the pre-pandemic status quo—the future of work has been fundamentally altered, and those who adapt thoughtfully will thrive in this new reality.
"""

summary = summarize(blog_post)
print(f'Summary: {summary}')

Input token count: 844
Generated token count: 150
Raw summary (with special tokens): </s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s></s>
Summary: 


In [56]:
dialogue_example = """
User: I need to reschedule my appointment for next week.
Agent: I'd be happy to help you with that. May I have your name and the current appointment date?
User: My name is Alex Smith and my appointment is for Monday at 2pm.
Agent: Thank you, Mr. Smith. I see your appointment here. What day and time would work better for you next week?
User: Would Thursday at 10am be available?
Agent: Let me check... Yes, Thursday at 10am is available. Would you like me to reschedule your appointment for that time?
User: Yes, please. And can you send me a confirmation email?
Agent: Absolutely. I've rescheduled your appointment for Thursday at 10am, and I'll send a confirmation email to the address we have on file. Is there anything else I can help you with today?
User: No, that's all. Thank you for your help.
Agent: You're welcome, Mr. Smith. We look forward to seeing you next Thursday at 10am. Have a great day!
"""

summary = summarize(dialogue_example)
print(f'Summary: {summary}')

Input token count: 232
Generated token count: 150
Raw summary (with special tokens): </s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s></s>
Summary: 
